In [ ]:
import numpy as np
import cv2
import pydicom
import pandas as pd
from keras.utils import to_categorical
from keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout, BatchNormalization, LeakyReLU, Multiply
from keras.applications import DenseNet201
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import tensorflow as tf
import os

# Создание путей до датасетов

In [ ]:
def dataframe_dcm(root_dir):
    """
    Функция для сбора информации о файлах .dcm в указанном каталоге и его подкаталогах.

    Аргументы:
    root_dir (str): Путь к корневому каталогу для поиска файлов.

    Возвращает:
    pd.DataFrame: DataFrame с информацией о пути к файлам и именами родительских директорий.
    """
    # Список для хранения данных о файлах .dcm
    data = []

    # Проход по всем подкаталогам и файлам
    for dirpath, _, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith('.dcm'):
                # Полный путь к файлу
                file_path = os.path.join(dirpath, filename)

                # Извлечение списка директорий в пути
                dirs = file_path.split(os.sep)

                # Извлечение нужного уровня родительской папки
                if len(dirs) >= 3:
                    # Проверка, что структура разрешает нам взять третий элемент
                    parent_folder = dirs[-3]
                else:
                    parent_folder = None

                # Добавление информации в список
                data.append({'path_name': parent_folder, 'file_path': file_path})

    # Создание DataFrame
    df = pd.DataFrame(data)

    # Добавление столбца index
    df.reset_index(inplace=True)

    return df


# Использование функции для медиц-й базы
medimp_directory = '/Users/vadimkirsanov/Desktop/MIPT_DS/Python_coding_data/chest_xray_hac/ds_medimp_train_good/block_0000_anon'
df_medimp = dataframe_dcm(medimp_directory)
df_medimp = df_medimp.sort_values(by='path_name')

# Использование функции для базы ключицы
clav_fracture_directory = '/Users/vadimkirsanov/Desktop/MIPT_DS/Python_coding_data/chest_xray_hac/ds_clav_fracture_train_good/block_0000_anon'
df_clav_fracture = dataframe_dcm(clav_fracture_directory)
df_clav_fracture = df_clav_fracture.sort_values(by='path_name')

# Печать результатов
print(df_medimp)
print(df_clav_fracture)

In [ ]:
# Определение путей к файлам Excel
medimp_excel_path = '/Users/vadimkirsanov/Desktop/MIPT_DS/Python_coding_data/chest_xray_hac/ds_medimp_train_good/block_0000_anon/block_0000_anon.xlsx'
clav_fracture_excel_path = '/Users/vadimkirsanov/Desktop/MIPT_DS/Python_coding_data/chest_xray_hac/ds_clav_fracture_train_good/block_0000_anon/block_0000_anon.xlsx'

# Чтение данных из файлов Excel в DataFrame
medimp_data = pd.read_excel(medimp_excel_path)
clav_fracture_data = pd.read_excel(clav_fracture_excel_path)

# Сортировка данных по столбцу `study_instance_anon`
medimp_data_sorted = medimp_data.sort_values(by='study_instance_anon')
clav_fracture_data_sorted = clav_fracture_data.sort_values(by='study_instance_anon')

# Вывод первых нескольких строк каждого отсортированного DataFrame
print("Medimp Data (Sorted):")
print(medimp_data_sorted.head())

print("\nClavicle Fracture Data (Sorted):")
print(clav_fracture_data_sorted.head())

In [ ]:
# Объединяем данные 'medimp_data_sorted' с 'df_medimp' на основе столбцов 'path_name' и 'study_instance_anon'
df_medimp = df_medimp.merge(
    medimp_data_sorted[['study_instance_anon', 'pathology']],
    how='left',
    left_on='path_name',
    right_on='study_instance_anon'
)

# Удаляем временный столбец 'study_instance_anon' после слияния
df_medimp.drop(columns=['study_instance_anon'], inplace=True)

# Удаляем временный столбец 'index', если он существует
df_medimp.drop(columns=['index'], inplace=True)

# Объединяем данные 'clav_fracture_data_sorted' с 'df_clav_fracture' на основе столбцов 'path_name' и 'study_instance_anon'
df_clav_fracture = df_clav_fracture.merge(
    clav_fracture_data_sorted[['study_instance_anon', 'pathology']],
    how='left',
    left_on='path_name',
    right_on='study_instance_anon'
)

# Удаляем временный столбец 'study_instance_anon' после слияния
df_clav_fracture.drop(columns=['study_instance_anon'], inplace=True)

# Удаляем временный столбец 'index', если он существует
df_clav_fracture.drop(columns=['index'], inplace=True)

# Выводим первые пять строк результата для проверки
print(df_medimp.head())
print(df_clav_fracture.head())

# Обучение модели

In [ ]:
# Параметры
IMG_SIZE = 224  # Размер изображения
BATCH_SIZE = 16  # Размер пакета
EPOCHS = 10  # Количество эпох

# Шаг 1: Загрузка данных DICOM
def load_dicom_images(df, img_size):
    # Функция для загрузки изображений DICOM и их предобработки
    images, labels = [], []
    for _, row in df.iterrows():
        dicom_path = row['file_path']
        label = row['pathology']
        dicom = pydicom.dcmread(dicom_path)  # Чтение файла DICOM
        img = dicom.pixel_array  # Извлечение массива пикселей
        # Изменение размера изображения
        img_resized = cv2.resize(img, (img_size, img_size))
        # Преобразование изображения в формат RGB
        img_resized = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)
        images.append(img_resized)
        labels.append(label)
    return np.array(images), np.array(labels)

# Загрузка изображений и меток
images_medimp, labels_medimp = load_dicom_images(df_medimp, IMG_SIZE)
images_clav, labels_clav = load_dicom_images(df_clav_fracture, IMG_SIZE)

# Объединение данных
images = np.concatenate([images_medimp, images_clav])
labels = np.concatenate([labels_medimp, labels_clav])

# Шаг 2: Подготовка данных
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Аугментация данных
datagen = ImageDataGenerator(
    rotation_range=15,  # Диапазон вращения
    width_shift_range=0.1,  # Диапазон сдвига по ширине
    height_shift_range=0.1,  # Диапазон сдвига по высоте
    horizontal_flip=True,  # Горизонтальное отражение
    rescale=1./255  # Нормализация
)

datagen.fit(X_train)

# Нормализация тестовых данных
X_test = X_test / 255.0

# Шаг 3: Построение модели
base_model = DenseNet201(
    weights='/Users/vadimkirsanov/Desktop/MIPT_DS/Python_coding_data/chest_xray_hac/saved_models/densenet201.keras', # укажите путь до актуальной модели
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

# Тонкая настройка базовой модели
for layer in base_model.layers:
    layer.trainable = False

# Создание дополнительных слоев для модели
dense = base_model.output
dense = GlobalAveragePooling2D()(dense)

# BatchNormalization и LeakyReLU
dense = BatchNormalization()(dense)
dense = Dense(512)(dense)
dense = LeakyReLU(alpha=0.1)(dense)

# Механизм внимания (attention)
attention_probs = Dense(512, activation='softmax', name='attention_probs')(dense)
attention_mul = Multiply()([dense, attention_probs])

# Слой Dropout
dense = Dropout(0.5)(attention_mul)

# Ещё один слой BatchNormalization и LeakyReLU
dense = BatchNormalization()(dense)
dense = Dense(256)(dense)
dense = LeakyReLU(alpha=0.1)(dense)

# Второй механизм внимания
attention_probs_2 = Dense(256, activation='softmax', name='attention_probs_2')(dense)
attention_mul_2 = Multiply()([dense, attention_probs_2])

# Второй слой Dropout
dense = Dropout(0.3)(attention_mul_2)

# Выходной слой
output = Dense(2, activation='softmax')(dense)

# Компиляция модели
model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Шаг 4: Обучение модели
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
    validation_data=(X_test, y_test),
    epochs=EPOCHS,
    steps_per_epoch=len(X_train) // BATCH_SIZE
)

# Шаг 5: Оценка модели
predictions = model.predict(X_test)
auc_score = roc_auc_score(y_test, predictions[:, 1])
print(f"AUC: {auc_score:.4f}")

# Шаг 6: Постобработка
def classify_output(predictions):
    results = []
    for pred in predictions:
        foreign_bodies_prob = pred[0]
        clavicle_fracture_prob = pred[1]
        if foreign_bodies_prob < 0.5 and clavicle_fracture_prob < 0.5:
            results.append("Normal")
        elif foreign_bodies_prob >= clavicle_fracture_prob:
            results.append("Foreign Bodies")
        else:
            results.append("Clavicle Fracture")
    return results

results = classify_output(predictions)
print("Classification Results:", results)

In [ ]:
model.save('my_model_3.keras')